In [1]:
# Authors: Jean-Remi King <jeanremi.king@gmail.com>
#          Alexandre Gramfort <alexandre.gramfort@telecom-paristech.fr>
#          Denis Engemann <denis.engemann@gmail.com>
#
# License: BSD (3-clause)

import numpy as np

import mne
from mne.datasets import sample
from mne.decoding import GeneralizationAcrossTime

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
# Preprocess data
data_path = sample.data_path()
# Load and filter data, set up epochs
raw_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'
events_fname = data_path + '/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
raw = mne.io.Raw(raw_fname, preload=True)
picks = mne.pick_types(raw.info, meg=True, exclude='bads')  # Pick MEG channels
raw.filter(1, 30, method='fft')  # Band pass filtering signals
events = mne.read_events(events_fname)
event_id = {'AudL': 1, 'AudR': 2, 'VisL': 3, 'VisR': 4}
decim = 2  # decimate to make the example faster to run
epochs = mne.Epochs(raw, events, event_id, -0.050, 0.400, proj=True,
                    picks=picks, baseline=None, preload=True,
                    reject=dict(mag=5e-12), decim=decim, verbose=False)




[........................................] 100.00000 / (1.35 GB / 1.35 GB)   
Verifying download hash.
Decompressing the archive: /volatile/anaconda/lib/python2.7/examples/MNE-sample-data-processed.tar.gz
(please be patient, this can take some time)
Opening raw data file /volatile/anaconda/lib/python2.7/examples/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
Current compensation grade : 0
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Reading 0 ... 41699  =      0.000 ...   277.709 secs...
[done]
Band-pass filtering from 1 - 30 Hz


In [3]:
# We will train the classifier on all left visual vs auditory trials
# and test on all right visual vs auditory trials.

# In this case, because the test data is independent from the train data,
# we test the classifier of each fold and average the respective predictions.

# Define events of interest
triggers = epochs.events[:, 2]
viz_vs_auditory = np.in1d(triggers, (1, 2)).astype(int)

gat = GeneralizationAcrossTime(predict_mode='mean-prediction', n_jobs=1)

# For our left events, which ones are visual?
viz_vs_auditory_l = (triggers[np.in1d(triggers, (1, 3))] == 3).astype(int)
# To make scikit-learn happy, we converted the bool array to integers
# in the same line. This results in an array of zeros and ones:
print("The unique classes' labels are: %s" % np.unique(viz_vs_auditory_l))



The unique classes' labels are: [0 1]


In [6]:
triggers

array([2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 2, 3, 1, 4, 2, 3, 1, 4,
       2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4,
       2, 3, 1, 4, 2, 3, 1, 4, 2, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4,
       2, 3, 1, 4, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 2,
       3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 4, 2, 3, 1, 4, 2,
       3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 1, 4, 2, 3, 1, 4, 2,
       3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 3, 1, 4, 2, 3, 1, 4, 3, 1, 4, 2, 3,
       1, 4, 2, 3, 1, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3,
       4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 3, 1,
       4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 2, 3, 1, 4, 2, 3, 1,
       4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1,
       4, 2, 3, 1, 4, 2, 3, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 1, 4,
       2, 3, 1, 4, 2, 1, 4, 2, 3, 1, 4], dtype=uint32)

In [ ]:
gat.fit(epochs[('AudL', 'VisL')], y=viz_vs_auditory_l)

# For our right events, which ones are visual?
viz_vs_auditory_r = (triggers[np.in1d(triggers, (2, 4))] == 4).astype(int)

gat.score(epochs[('AudR', 'VisR')], y=viz_vs_auditory_r)
gat.plot(
    title="Generalization Across Time (visual vs auditory): left to right")